In [5]:
# !pip install datasets
from datasets import load_dataset

ds = load_dataset('FunDialogues/customer-service-grocery-cashier')

c:\kiosks_\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
ds.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        100 non-null    int64 
 1   id           100 non-null    int64 
 2   description  100 non-null    object
 3   dialogue     100 non-null    object
dtypes: int64(2), object(2)
memory usage: 3.3+ KB


In [10]:
ds[:2]

,index,id,description,dialogue
0,0,1,Price inquiry,"Customer: Excuse me, could you tell me the pri..."
1,1,2,Product availability,Cashier: Good afternoon! How can I assist you ...


In [13]:
print(len(ds['train']))

100


In [7]:
ds.column_names

{'train': ['index', 'id', 'description', 'dialogue']}

In [15]:
# Examine a few examples
print(ds["train"][0])
print(ds["train"][2])

{'index': 0, 'id': 1, 'description': 'Price inquiry', 'dialogue': 'Customer: Excuse me, could you tell me the price of the apples per pound? Cashier: Certainly! The price for the apples is $1.99 per pound.'}
{'index': 2, 'id': 3, 'description': 'Coupon usage', 'dialogue': 'Customer: I have this coupon for 20% off. Can I use it on this purchase? Cashier: Of course! Let me scan it for you. Great, your total comes to $32.50 after applying the discount.'}


In [22]:
dataset.to_csv('data_set.csv', index=False)

Data reading and preprocessing 

In [2]:
import os 
os.chdir('C:/Kiosks_')
os.getcwd()

'C:\\Kiosks_'

In [28]:
import pandas
from main.components.data_preprocessing.config.data_preproc_config import data_preproc_configs

obj = data_preproc_configs()
configs = obj.data_preprocessing_configs()

os.makedirs(configs['raw_dataset_path'] , exist_ok=True)
# ds = ds['train'].to_pandas()
ds.to_csv(os.path.join(configs['raw_dataset_path'], 'raw_data.csv'), index = False)
# print(configs)

created directory at: artifacts/datasets


In [30]:
data = pandas.read_csv(os.path.join(configs['raw_dataset_path'], 'raw_data.csv'))
data.head(5)

,index,id,description,dialogue
0,0,1,Price inquiry,"Customer: Excuse me, could you tell me the pri..."
1,1,2,Product availability,Cashier: Good afternoon! How can I assist you ...
2,2,3,Coupon usage,Customer: I have this coupon for 20% off. Can ...
3,3,4,Return policy,Cashier: How may I help you? Customer: I bough...
4,4,5,Payment method,"Customer: Excuse me, can I pay with a credit c..."


In [31]:
data['dialogue'][3]
data.drop('index', axis=1, inplace=True)

In [38]:
import json

# Assuming `data` is your DataFrame
json_entries = []

for i in range(len(data)):  # Looping through rows
    instruction = data.loc[i, 'description']
    dialogs = data.loc[i, 'dialogue']

    json_entry = {
        'instruction': instruction,
        'input': dialogs
    }
    json_entries.append(json_entry)

# Write all JSON entries to a single file
with open('artifacts/raw_datasets/Sample.jsonl', 'w') as f:
    f.write('\n'.join([json.dumps(entry) for entry in json_entries]))


In [62]:
import re
with open('artifacts/raw_datasets/Sample.jsonl', 'w') as f:
    for i in range(len(data['dialogue'])):
        diaglog = data['dialogue'][i]

        parts = re.split(r'(Cashier:|Customer:)', diaglog)[1:]
        # print(parts)
        # print(len(parts))
        messages = []

        for j in range(0, len(parts), 2):
            if j+1 < len(parts):
                role = parts[j].strip()
                content = parts[j+1].strip()
                # Map roles to user/assistant format
                if role == "Customer:":
                    messages.append({"role": "user", "content": content})
                elif role == "Cashier:":
                    messages.append({"role": "assistant", "content": content})
        
        # Create JSON entry for chat format
        json_entry = {
            "messages": messages
        }
        
        # Write to file
        f.write(json.dumps(json_entry) + '\n')


In [ ]:
parts


['Customer:',
 ' Excuse me, can I pay with a credit card? ',
 'Cashier:',
 " Absolutely! We accept all major credit cards. Please insert or swipe your card when you're ready."]

Note The diversity in conversation

In [ ]:
{"messages": [{"role": "assistant", "content": "I see this item is marked as 'on sale.' Can you tell me the discounted price?"}, {"role": "user", "content": "Sure! The sale price is {item_price}, marked down from {item_discount_price}."}]}

{"messages": [{"role": "user", "content": "Are there any discounts available for senior citizens?"}, {"role": "assistant", "content": "Yes, we offer a {discount_percentage} for customers aged 60 and above. Just show your ID at the checkout."}]}

{"messages": [{"role": "assistant", "content": "I have this coupon for $5 off. Can I use it on this purchase?"}, {"role": "user", "content": "I'm sorry, but this coupon is only valid for purchases over $50. Your total is $30."}]}
# 👇👇
{"messages": [{"role": "assistant", "content": "I have this coupon for {off_amount} off. Can I use it on this purchase?"}, {"role": "user", "content": "I'm sorry, but this coupon is only valid for purchases over {min_price_for_dis}. Your total is {user_total}."}]}

{"messages": [{"role": "assistant", "content": "Good afternoon! How can I assist you today?"}, {"role": "user", "content": "Hi, do you have any {item_name} ?"}, {"role": "assistant", "content": "Yes, we do. It's located in {category}, right next to the organic section."}]}

{"messages": [{"role": "user", "content": "Are there any special deals or promotions this week?"}, {"role": "assistant", "content": "Yes, we have a {special_deals} and we have promotions offer of {promotions}"}]}

{"messages": [{"role": "user", "content": "I have this coupon for {off_amount} off. Can I use it on this purchase?"}, {"role": "assistant", "content": "Of course! Let me scan it for you. Great, your total comes to {amout_after_off} after applying the discount."}]}

# from 101 extras
{"messages": [{"role": "user", "content": "Hello, do you have more items in the {category}?"}, {"role": "assistant", "content": "Let me check... Yes, we have {many_items} available in the back. I can fetch one for you."}]}

{"messages": [{"role": "user", "content": "Could I get a {receipt_type} receipt?"}, {"role": "assistant", "content": "Of course! I'll email it to you. Would you like a printed copy too?"}]}

{"messages": [{"role": "assistant", "content": "Today we have a special on {item_category}: buy one, get one {discount_type}!"}, {"role": "user", "content": "Great! I'll take two {item_name}s."}, {"role": "assistant", "content": "Your total savings are {savings_amount}."}]}
{"messages": [{"role": "user", "content": "Can you recommend a good {category}?"}, {"role": "assistant", "content": "Our {recommended_item} is popular. It’s {reason}."}]}

{"messages": [{"role": "user", "content": "I need a {bag_type} bag."}, {"role": "assistant", "content": "Sure! That’ll be an additional {cost}. I’ll add it to your total."}]}

{"messages": [{"role": "user", "content": "What's in the {item_name} meal?"}, {"role": "assistant", "content": "It comes with a {main_item}, {side_item}, {drink_size} and a {description}."}, {"role": "user", "content": "Can I swap the fries for a salad?"}, {"role": "assistant", "content": "Yes, there’s no extra charge for that substitution."}, {"role": "user", "content": "Perfect. I'll take the meal then."}, {"role": "assistant", "content": "Would you like to add a dessert for {dessert_price}?"}, {"role": "user", "content": "No, I'm good. Thanks!"}]}

{"messages": [{"role": "assistant", "content": "Good afternoon! Would you like a table for how many?"}, {"role": "user", "content": "Table for two, please."}, {"role": "assistant", "content": "Right this way. Here's the menu."}, {"role": "user", "content": "Thank you. What’s your recommendation?"}, {"role": "assistant", "content": "Our {menu_special} is a favorite."}, {"role": "user", "content": "I’ll have that."}, {"role": "assistant", "content": "Excellent choice! I'll put that in."}]}

# Removed conv json
{"messages": [{"role": "assistant", "content": "How may I help you?"}, {"role": "user", "content": "I bought this blender yesterday, but it's not working properly. Can I return it?"}, {"role": "assistant", "content": "I'm sorry to hear that. Yes, you can bring it back with the receipt within 30 days for a refund or exchange."}]}

{"messages": [{"role": "user", "content": "What time do you close today?"}, {"role": "assistant", "content": "We close at {closing_time}. You still have a couple of hours to shop."}]}

In [ ]:

# Coversation variables
{closing_time},
{opening_time},
{todays_offers}
{membership_description}

opening artifacts\cashier_conversations_placeholders_50.jsonl file and editing {item_name_1} (series of item name) to fixed {item_name}

In [2]:
import os
os.getcwd()
os.chdir('C:/Kiosks_')

In [ ]:

with open('artifacts\\cashier_conversations_placeholders_50.jsonl', 'r') as f:
    # for line in f:
    #     target_word = line.strip()
    #     if target_word in ['{item_name_1}', '{item_name_2}', '{item_name_3}']:
    #         item_name = target_word.replace('_1', '').replace('_2', '').replace('_3', '')
    #         # print(target_word
    lines= f.readlines()
    modified_one = []
    for line in lines:
        modified_one.append(line.replace('{item_name_1}', '{item_name}').replace('{item_name_2}', '{item_name}').replace('{item_name_3}', '{item_name}'))

#     Instead of overwriting modified_one each time, store all modified lines in a list and write them together.
#     modified_lines = [
#     line.replace('{item_name_1}', '{item_name}')
#         .replace('{item_name_2}', '{item_name}')
#         .replace('{item_name_3}', '{item_name}')
#     for line in lines
# ]
        
with open('artifacts\\cashier_conversations_placeholders_50.jsonl', 'w') as f:
            f.writelines(modified_one)

print('Sucessfully replaced the item_name_1, item_name_2 AND item_name_3 with item_name')

Sucessfully replaced the item_name_1, item_name_2 AND item_name_3 with item_name


Editing for diverse conv like item_name_(rand_num)

In [17]:
import re

with open('artifacts\\raw_datasets\\diverse_cashier_conversations_50.jsonl', 'r') as f:
    modified_lines = [re.sub(r'\{item_name_\d+\}', '{item_name}', line) for line in lines]
        
with open('artifacts\\raw_datasets\\10_diverse_samples.jsonl', 'w') as f:
            f.writelines(modified_one)

print('Sucessfully replaced the item_name_1, item_name_2 AND item_name_3 with item_name')

Sucessfully replaced the item_name_1, item_name_2 AND item_name_3 with item_name


In [7]:
import json
with open('artifacts\\datasets\\Dataset.jsonl', 'r')as f:
    lines = [json.loads(line.strip()) for line in f]
    lines

In [18]:
from datasets import Dataset

def convert_to_prompt(example):
    messages = example["messages"]
    prompt = ""
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            prompt += f"<|user|>\n{content}\n"
        else:
            prompt += f"<|assistant|>\n{content}\n"
    return {"text": prompt}

processed_data = [convert_to_prompt(ex) for ex in data]
dataset = Dataset.from_list(processed_data)

In [19]:
dataset

Dataset({
    features: ['text'],
    num_rows: 136
})

In [20]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3b-it",  # You can try 4b when it's available
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!